# Fase 2 - Padronização de colunas e tipos (schema)
Agora que já possuímos os dados armazenados como Delta tables internamente e já realizamos uma etapa prévia de EDA (Análise Exploratória), iremos iniciar o tratamento da camada Silver padronizando tipos, nomes e dados das colunas.

In [0]:
# Importando nossos dados como DataFrames para trabalharmos com as colunas e seus dados.

df_crabpotandothercatchables_silver = spark.read.table("stardew_project.bronze.crabpotandothercatchables")

df_fish_detail_silver = spark.read.table("stardew_project.bronze.fish_detail")

df_fish_price_breakdown_silver = spark.read.table("stardew_project.bronze.fish_price_breakdown")

df_legendary_fish_detail_silver = spark.read.table("stardew_project.bronze.legendary_fish_detail")

df_legendary_fish_price_breakdown_silver = spark.read.table("stardew_project.bronze.legendary_fish_price_breakdown")

df_legendaryfishII_silver = spark.read.table("stardew_project.bronze.legendaryfishii")

df_nightmarketfish_silver = spark.read.table("stardew_project.bronze.nightmarketfish")

df_villagers_silver = spark.read.table("stardew_project.bronze.villagers")

df_behavior_silver = spark.read.table("stardew_project.bronze.behavior")

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql.functions import col, split
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import instr

# Funções auxiliares
Criação de funções auxiliares para facilitar o processo de padronização posterior aplicado a diversos DataFrames.

In [0]:
# Função auxiliar rename_cols, que renomeia e padroniza caracteres especiais para lower_snake_case as colunas dos DataFrames, além de adicionar uma coluna "ingestion_timestamp" para fins de auditoria posterior.

import re

def rename_cols(df):

  origin_names = list(df.columns)

  names_to_replace = {}

  for column in origin_names:
    names_to_replace[column] = re.sub(r"[^A-Za-z0-9]+", "_", column.lower())

  for key, value in names_to_replace.items():
    names_to_replace[key] = re.sub(r"^_+|_+$", "", value)
  
  df = df.withColumnsRenamed(names_to_replace)
  return df.withColumn("ingestion_timestamp", current_timestamp())

In [0]:
# Função split_col, que extrai a primeira ou segunda posição de uma coluna e a retorna como ela mesma.

def split_col(df, column: str, position: int, sep: str, cast: str, alias: str | None = None):
    if alias is None:
        df = df.withColumn(column, split(col(column), sep)[position].cast(cast))
    else:
        df = df.withColumn(alias, split(col(column), sep)[position].cast(cast))
    return df

In [0]:

# Função normalize_column, que normaliza unicodes dos valores das linhas da coluna passada como argumento para "-".

def normalize_column(df, column: str):
    if df.schema[column].dataType == StringType():
        df = df.withColumn(column, f.regexp_replace(col(column), r"\p{Pd}", "-"))
        df = df.withColumn(column, f.regexp_replace(col(column), r"\s*-\s*", "-"))
    else:
        pass

    return df

In [0]:
# Função min_and_max, que extrai valores mínimos e máximos das colunas e as transforma em colunas distintas.

def min_and_max(df, column: str, sep: str, cast: str, drop: bool):
    col_min = column + "_min"
    col_max = column + "_max"

    df = df.withColumns({
        col_min: f.when(col(column).contains(sep), split(col(column), sep)[0].cast(cast)).otherwise(col(column).cast(cast)),
        col_max: f.when(col(column).contains(sep), split(col(column), sep)[1].cast(cast)).otherwise(col(column).cast(cast))
    })

    if drop == True:
        df = df.drop(column)
    else:
        pass
    return df

In [0]:
# Função split_time, que extrai valores mínimos e máximos da coluna time e as transforma em colunas distintas, mantendo como string.

def split_time(df, column: str, sep: str, drop: bool):
    col_min = column + "_min"
    col_max = column + "_max"

    df = df.withColumns({
        col_min: f.when(col(column).contains(sep), split(col(column), sep)[0]).otherwise(col(column)),
        col_max: f.when(col(column).contains(sep), split(col(column), sep)[1]).otherwise(col(column))
    })

    if drop == True:
        df = df.drop(column)
    else:
        pass
    return df

In [0]:
# Função normalize_prices, que serve para normalizar as colunas de preço dos DataFrames e fazer o casting corretamente de cada um dos casos.

def normalize_prices(df, cols_to_skip: list):
    all_cols = df.columns
    cols_to_str = ["price", "fish_profession_25", "fish_profession_50", "fisher_profession_25", "angler_profession_50"]

    for column in all_cols:
      if column in cols_to_skip:
        continue
      else:
        df = df.withColumn(column, f.regexp_replace(col(column), r"(?<=\d),(?=\d)|(?<=\d)g(?=[,\s]|$)", ""))

    for column in all_cols:
      if column not in cols_to_str and column not in cols_to_skip:
        df = df.withColumn(column, col(column).cast("integer"))
      else:
         df = df.withColumn(column, col(column).cast("string"))

    return df


In [0]:
# Função reordering_df, que serve para reordenarmos as colunas dos DataFrames de volta para suas colocações originais (retirando as adições de colunas do final).

def reordering_df(df, old_columns):
    new_columns = df.columns
    final_order = []

    replaces = {
        "difficulty_behavior": ["difficulty", "behavior"],
        "size_inches": ["size_inches_min", "size_inches_max"],
        "time": ["time_min", "time_max"]
    }

    for old_col in old_columns:
        if old_col in new_columns:
            final_order.append(old_col)
        elif old_col in replaces.keys():
            final_order.append(replaces[old_col][0])
            final_order.append(replaces[old_col][1])
        elif old_col + "_min" in new_columns:
            final_order.append(old_col + "_min")
            final_order.append(old_col + "_max")
        else:
            pass

    df = df.select(final_order)
    return df

In [0]:
# Função save_df, que serve para salvar os DataFrames finais como tabelas Delta fazendo schema enforcement e com schema evolution ativado para toda a sessão.

def save_df(df, table: str, key: str):
    view_name = (table + "df")
    df.createOrReplaceTempView(view_name)

    df_schema = df.schema

    map_types = {}
    for tipo in df_schema:
        map_types[tipo.name] = tipo.dataType.simpleString()

    final_schema = ""
    for name, tipo in map_types.items():
        final_schema += name + " " + tipo + ", "

    final_schema = final_schema.rstrip(", ")
    final_schema = "(" + final_schema + ")"

    if spark.catalog.tableExists(f"stardew_project.silver.{table}"):
        print(f"tabela {table} existe")
        spark.sql(f"""
            MERGE INTO stardew_project.silver.{table} AS tgt
            USING {view_name} AS src
            ON tgt.{key} = src.{key}
            WHEN MATCHED THEN UPDATE SET *
            WHEN NOT MATCHED THEN INSERT *
        """)
    else:
        print(f"tabela {table} não existe")
        spark.sql(f"""
                  CREATE TABLE stardew_project.silver.{table}
                  {final_schema}
                  USING DELTA 
                  AS SELECT * FROM {view_name}""")
        
    spark.sql(f"DROP VIEW {view_name}")

# Padronizando colunas: df_crabpotandothercatchables_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_crabpotandothercatchables_silver = rename_cols(df_crabpotandothercatchables_silver)
crab_origin_cols = df_crabpotandothercatchables_silver.columns

df_crabpotandothercatchables_silver.limit(5).display()

name,description,location,trap_chance_non_mariner,trap_chance_mariner,size_inches,used_in,ingestion_timestamp
Lobster,A large ocean-dwelling crustacean with a strong tail.,Ocean,4%,14%,2-20,"""Lobster Bisque """"Wanted Lobster"""" Quest Elliott (loved Gift)""",2025-11-25T23:16:26.419Z
Clam,Someone lived here once.,Ocean,9%,14%,1-5,Chowder Speed-Gro,2025-11-25T23:16:26.419Z
Crayfish,A small freshwater relative of the lobster.,Freshwater,21%,33%,1-8,Fish Stew,2025-11-25T23:16:26.419Z
Crab,A marine crustacean with two powerful pincers.,Ocean,8%,14%,1-20,Crab Cakes,2025-11-25T23:16:26.419Z
Cockle,A common saltwater clam.,Ocean,12%,14%,1-5,null,2025-11-25T23:16:26.419Z


In [0]:
# Padronização de tipos e desempacotamento (adição) de colunas.

df_crabpotandothercatchables_silver = split_col(df_crabpotandothercatchables_silver, "trap_chance_non_mariner", 0, "%", "int")
df_crabpotandothercatchables_silver = split_col(df_crabpotandothercatchables_silver, "trap_chance_mariner", 0, "%", "int")
df_crabpotandothercatchables_silver = min_and_max(df_crabpotandothercatchables_silver, "size_inches", "-", "int", True)
df_crabpotandothercatchables_silver = reordering_df(df_crabpotandothercatchables_silver, crab_origin_cols)

df_crabpotandothercatchables_silver.limit(5).display()

name,description,location,trap_chance_non_mariner,trap_chance_mariner,size_inches_min,size_inches_max,used_in,ingestion_timestamp
Lobster,A large ocean-dwelling crustacean with a strong tail.,Ocean,4,14,2,20,"""Lobster Bisque """"Wanted Lobster"""" Quest Elliott (loved Gift)""",2025-11-25T23:16:31.892Z
Clam,Someone lived here once.,Ocean,9,14,1,5,Chowder Speed-Gro,2025-11-25T23:16:31.892Z
Crayfish,A small freshwater relative of the lobster.,Freshwater,21,33,1,8,Fish Stew,2025-11-25T23:16:31.892Z
Crab,A marine crustacean with two powerful pincers.,Ocean,8,14,1,20,Crab Cakes,2025-11-25T23:16:31.892Z
Cockle,A common saltwater clam.,Ocean,12,14,1,5,null,2025-11-25T23:16:31.892Z


# Padronizando colunas: df_fish_detail_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_fish_detail_silver = rename_cols(df_fish_detail_silver)
df_fish_detail_silver = normalize_column(df_fish_detail_silver, "time")
fishdetail_origin_cols = df_fish_detail_silver.columns

df_fish_detail_silver.limit(5).display()

name,description,location,time,season,weather,size_inches,difficulty_behavior,base_xp,used_in,ingestion_timestamp
Pufferfish,Inflates when threatened.,"Ocean, Ginger Island oceans",12pm-4pm,"Summer, (All Seasons on Ginger Island)",Sun,1-37,80 floater,29,"""Specialty Fish Bundle Abigail (loved gift) """"Aquatic Research"""" Quest""",2025-11-25T23:16:33.064Z
Anchovy,A small silver fish found in the ocean.,Ocean,Anytime,"Spring, Fall",Any,1-17,30 dart,13,null,2025-11-25T23:16:33.064Z
Tuna,A large fish that lives in the ocean.,"Ocean, Ginger Island oceans",6am-7pm,"Summer, Winter, (All Seasons on Ginger Island)",Any,12-61,70 smooth,26,"Ocean Fish Bundle, Quality Fish Bundle Fish Taco",2025-11-25T23:16:33.064Z
Sardine,A common ocean fish.,Ocean,6am-7pm,"Spring, Fall, Winter",Any,1-13,30 dart,13,Ocean Fish Bundle Dish O' The Sea,2025-11-25T23:16:33.064Z
Bream,A fairly common river fish that becomes active at night.,River (Town+Forest),6pm-2am,All Seasons,Any,12-31,35 smooth,14,Night Fishing Bundle Baked Fish,2025-11-25T23:16:33.064Z


In [0]:
# Padronização de tipos e desempacotamento (adição) de colunas.

df_fish_detail_silver = split_col(df_fish_detail_silver, "difficulty_behavior", 0, " ", "int", "difficulty")
df_fish_detail_silver = split_col(df_fish_detail_silver, "difficulty_behavior", 1, " ", "string", "behavior").drop("difficulty_behavior")
df_fish_detail_silver = min_and_max(df_fish_detail_silver, "size_inches", "-", "int", True)
df_fish_detail_silver = split_time(df_fish_detail_silver, "time", "-", True)
df_fish_detail_silver = reordering_df(df_fish_detail_silver, fishdetail_origin_cols)

df_fish_detail_silver.limit(5).display()

name,description,location,time_min,time_max,season,weather,size_inches_min,size_inches_max,difficulty,behavior,base_xp,used_in,ingestion_timestamp
Pufferfish,Inflates when threatened.,"Ocean, Ginger Island oceans",12pm,4pm,"Summer, (All Seasons on Ginger Island)",Sun,1,37,80,floater,29,"""Specialty Fish Bundle Abigail (loved gift) """"Aquatic Research"""" Quest""",2025-11-25T23:16:34.324Z
Anchovy,A small silver fish found in the ocean.,Ocean,Anytime,Anytime,"Spring, Fall",Any,1,17,30,dart,13,null,2025-11-25T23:16:34.324Z
Tuna,A large fish that lives in the ocean.,"Ocean, Ginger Island oceans",6am,7pm,"Summer, Winter, (All Seasons on Ginger Island)",Any,12,61,70,smooth,26,"Ocean Fish Bundle, Quality Fish Bundle Fish Taco",2025-11-25T23:16:34.324Z
Sardine,A common ocean fish.,Ocean,6am,7pm,"Spring, Fall, Winter",Any,1,13,30,dart,13,Ocean Fish Bundle Dish O' The Sea,2025-11-25T23:16:34.324Z
Bream,A fairly common river fish that becomes active at night.,River (Town+Forest),6pm,2am,All Seasons,Any,12,31,35,smooth,14,Night Fishing Bundle Baked Fish,2025-11-25T23:16:34.324Z


# Padronizando colunas: df_fish_price_breakdown_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_fish_price_breakdown_silver = rename_cols(df_fish_price_breakdown_silver)
fish_price_breakdown_origin_cols = df_fish_price_breakdown_silver.columns

df_fish_price_breakdown_silver.limit(5).display()

name,pufferfish,anchovy,tuna,sardine,bream,largemouth_bass,smallmouth_bass,rainbow_trout,salmon,walleye,perch,carp,catfish,pike,sunfish,red_mullet,herring,eel,octopus,red_snapper,squid,sea_cucumber,super_cucumber,ghostfish,stonefish,ice_pip,lava_eel,sandfish,scorpion_carp,flounder,midnight_carp,sturgeon,tiger_trout,bullhead,tilapia,chub,dorado,albacore,shad,lingcod,halibut,woodskip,void_salmon,slimejack,stingray,lionfish,blue_discus,goby,ingestion_timestamp
Base Price,200g,30g,100g,40g,45g,100g,50g,65g,75g,105g,55g,30g,200g,100g,30g,75g,30g,85g,150g,50g,80g,75g,250g,45g,300g,500g,700g,75g,150g,100g,150g,200g,150g,75g,75g,50g,100g,75g,60g,120g,80g,75g,150g,100g,180g,100g,120g,150g,2025-11-25T23:16:35.271Z
BP Silver,250g,37g,125g,50g,56g,125g,62g,81g,93g,131g,68g,37g,250g,125g,37g,93g,37g,106g,187g,62g,100g,93g,312g,56g,375g,625g,875g,93g,187g,125g,187g,250g,187g,93g,93g,62g,125g,93g,75g,150g,100g,93g,187g,125g,225g,125g,150g,187g,2025-11-25T23:16:35.271Z
BP Gold,300g,45g,150g,60g,67g,150g,75g,97g,112g,157g,82g,45g,300g,150g,45g,112g,45g,127g,225g,75g,120g,112g,375g,67g,450g,750g,"1,050g",112g,225g,150g,225g,300g,225g,112g,112g,75g,150g,112g,90g,180g,120g,112g,225g,150g,270g,150g,180g,225g,2025-11-25T23:16:35.271Z
BP Irridium,400g,60g,200g,80g,90g,200g,100g,130g,150g,210g,110g,60g,400g,200g,60g,150g,60g,170g,300g,100g,160g,150g,500g,90g,600g,"1,000g","1,400g",150g,300g,200g,300g,400g,300g,150g,150g,100g,200g,150g,120g,240g,160g,150g,300g,200g,360g,200g,240g,300g,2025-11-25T23:16:35.271Z
Fisher Profession (+25%),250g,37g,125g,50g,56g,125g,62g,81g,93g,131g,68g,37g,250g,125g,37g,93g,37g,106g,187g,62g,100g,93g,312g,56g,375g,625g,875g,93g,187g,125g,187g,250g,187g,93g,93g,62g,125g,93g,75g,150g,100g,93g,187g,125g,225g,125g,150g,187g,2025-11-25T23:16:35.271Z


In [0]:
# Padronização de tipos e desempacotamento (adição) de colunas.

df_fish_price_breakdown_silver = normalize_prices(df_fish_price_breakdown_silver, ["name", "ingestion_timestamp"])
df_fish_price_breakdown_silver = reordering_df(df_fish_price_breakdown_silver, fish_price_breakdown_origin_cols)

df_fish_price_breakdown_silver.limit(5).display()

name,pufferfish,anchovy,tuna,sardine,bream,largemouth_bass,smallmouth_bass,rainbow_trout,salmon,walleye,perch,carp,catfish,pike,sunfish,red_mullet,herring,eel,octopus,red_snapper,squid,sea_cucumber,super_cucumber,ghostfish,stonefish,ice_pip,lava_eel,sandfish,scorpion_carp,flounder,midnight_carp,sturgeon,tiger_trout,bullhead,tilapia,chub,dorado,albacore,shad,lingcod,halibut,woodskip,void_salmon,slimejack,stingray,lionfish,blue_discus,goby,ingestion_timestamp
Base Price,200,30,100,40,45,100,50,65,75,105,55,30,200,100,30,75,30,85,150,50,80,75,250,45,300,500,700,75,150,100,150,200,150,75,75,50,100,75,60,120,80,75,150,100,180,100,120,150,2025-11-25 23:16:38.261289
BP Silver,250,37,125,50,56,125,62,81,93,131,68,37,250,125,37,93,37,106,187,62,100,93,312,56,375,625,875,93,187,125,187,250,187,93,93,62,125,93,75,150,100,93,187,125,225,125,150,187,2025-11-25 23:16:38.261289
BP Gold,300,45,150,60,67,150,75,97,112,157,82,45,300,150,45,112,45,127,225,75,120,112,375,67,450,750,1050,112,225,150,225,300,225,112,112,75,150,112,90,180,120,112,225,150,270,150,180,225,2025-11-25 23:16:38.261289
BP Irridium,400,60,200,80,90,200,100,130,150,210,110,60,400,200,60,150,60,170,300,100,160,150,500,90,600,1000,1400,150,300,200,300,400,300,150,150,100,200,150,120,240,160,150,300,200,360,200,240,300,2025-11-25 23:16:38.261289
Fisher Profession (+25%),250,37,125,50,56,125,62,81,93,131,68,37,250,125,37,93,37,106,187,62,100,93,312,56,375,625,875,93,187,125,187,250,187,93,93,62,125,93,75,150,100,93,187,125,225,125,150,187,2025-11-25 23:16:38.261289


# Padronizando colunas: df_legendary_fish_detail_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_legendary_fish_detail_silver = rename_cols(df_legendary_fish_detail_silver)
df_legendary_fish_detail_silver = normalize_column(df_legendary_fish_detail_silver, "time")
legendary_fish_detail_origin_cols = df_legendary_fish_detail_silver.columns

df_legendary_fish_detail_silver.limit(5).display()

name,description,location,time,season,weather,size_inches,difficulty_behavior,base_xp,ingestion_timestamp
Crimsonfish,Lives deep in the ocean but likes to lay its eggs in the warm summer water.,East Pier on The Beach. Requires level 5 fishing.,Any,Summer,Any,19-21,95 mixed,170,2025-11-25T23:16:39.595Z
Angler,Uses a bioluminescent dangler to attract prey.,North of JojaMart on the wooden plank bridge. Requires level 3 fishing.,Any,Fall,Any,17-19,85 smooth,155,2025-11-25T23:16:39.595Z
Legend,The king of all fish! They said he'd never be caught.,The Mountain Lake near the log. Requires level 10 fishing.,Any,Spring,Rain,49-51,110 mixed,195,2025-11-25T23:16:39.595Z
Glacierfish,Builds a nest on the underside of glaciers.,South end of Arrowhead Island in Cindersap Forest. Requires level 6 fishing.,Any,Winter,Any,26-28,100 mixed,180,2025-11-25T23:16:39.595Z
Mutant Carp,The strange waters of the sewer turned this carp into a monstrosity.,The Sewers,Any,All Seasons,Any,35-37,80 dart,145,2025-11-25T23:16:39.595Z


In [0]:
# Padronização de tipos e desempacotamento (adição) de colunas.

df_legendary_fish_detail_silver = split_col(df_legendary_fish_detail_silver, "difficulty_behavior", 0, " ", "int", "difficulty")
df_legendary_fish_detail_silver = split_col(df_legendary_fish_detail_silver, "difficulty_behavior", 1, " ", "string", "behavior").drop("difficulty_behavior")
df_legendary_fish_detail_silver = min_and_max(df_legendary_fish_detail_silver, "size_inches", "-", "int", True)
df_legendary_fish_detail_silver = split_time(df_legendary_fish_detail_silver, "time", "-", True)
df_legendary_fish_detail_silver = reordering_df(df_legendary_fish_detail_silver, legendary_fish_detail_origin_cols)

df_legendary_fish_detail_silver.limit(5).display()

name,description,location,time_min,time_max,season,weather,size_inches_min,size_inches_max,difficulty,behavior,base_xp,ingestion_timestamp
Crimsonfish,Lives deep in the ocean but likes to lay its eggs in the warm summer water.,East Pier on The Beach. Requires level 5 fishing.,Any,Any,Summer,Any,19,21,95,mixed,170,2025-11-25T23:16:40.773Z
Angler,Uses a bioluminescent dangler to attract prey.,North of JojaMart on the wooden plank bridge. Requires level 3 fishing.,Any,Any,Fall,Any,17,19,85,smooth,155,2025-11-25T23:16:40.773Z
Legend,The king of all fish! They said he'd never be caught.,The Mountain Lake near the log. Requires level 10 fishing.,Any,Any,Spring,Rain,49,51,110,mixed,195,2025-11-25T23:16:40.773Z
Glacierfish,Builds a nest on the underside of glaciers.,South end of Arrowhead Island in Cindersap Forest. Requires level 6 fishing.,Any,Any,Winter,Any,26,28,100,mixed,180,2025-11-25T23:16:40.773Z
Mutant Carp,The strange waters of the sewer turned this carp into a monstrosity.,The Sewers,Any,Any,All Seasons,Any,35,37,80,dart,145,2025-11-25T23:16:40.773Z


# Padronizando colunas: df_legendary_fish_price_breakdown_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_legendary_fish_price_breakdown_silver = rename_cols(df_legendary_fish_price_breakdown_silver)
df_legendary_fish_price_breakdown_origin_cols = df_legendary_fish_price_breakdown_silver.columns

df_legendary_fish_price_breakdown_silver.limit(5).display()

name,crimsonfish,angler,legend,glacierfish,mutant_carp,ingestion_timestamp
Base Price,"1,500g",900g,"5,000g","1,000g","1,000g",2025-11-25T23:16:41.762Z
BP Silver,"1,875g","1,125g","6,250g","1,250g","1,250g",2025-11-25T23:16:41.762Z
BP Gold,"2,250g","1,350g","7,500g","1,500g","1,500g",2025-11-25T23:16:41.762Z
BP Irridium,"3,000g","1,800g","10,000g","2,000g","2,000g",2025-11-25T23:16:41.762Z
Fisher Profession (+25%),"2,250g","1,350g","7,500g","1,500g","1,500g",2025-11-25T23:16:41.762Z


In [0]:
# Padronização de tipos e desempacotamento (adição) de colunas.

df_legendary_fish_price_breakdown_silver = normalize_prices(df_legendary_fish_price_breakdown_silver, ["name", "ingestion_timestamp"])
df_legendary_fish_price_breakdown_silver = reordering_df(df_legendary_fish_price_breakdown_silver, df_legendary_fish_price_breakdown_origin_cols)

df_legendary_fish_price_breakdown_silver.limit(5).display()

name,crimsonfish,angler,legend,glacierfish,mutant_carp,ingestion_timestamp
Base Price,1500,900,5000,1000,1000,2025-11-25 23:16:42.933551
BP Silver,1875,1125,6250,1250,1250,2025-11-25 23:16:42.933551
BP Gold,2250,1350,7500,1500,1500,2025-11-25 23:16:42.933551
BP Irridium,3000,1800,10000,2000,2000,2025-11-25 23:16:42.933551
Fisher Profession (+25%),2250,1350,7500,1500,1500,2025-11-25 23:16:42.933551


# Padronizando colunas: df_legendaryfishII_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_legendary_fishII_detail_silver = rename_cols(df_legendaryfishII_silver)
df_legendary_fishII_detail_silver = normalize_column(df_legendary_fishII_detail_silver, "time")
df_legendary_fishII_detail_origin_cols = df_legendary_fishII_detail_silver.columns

df_legendary_fishII_detail_silver.limit(5).display()

name,description,price,fisher_profession_25,angler_profession_50,location,time,season,weather,size_inches,difficulty_behavior,base_xp,ingestion_timestamp
Son of Crimsonfish,"He hatched in the warm summer water, and followed in the footsteps of his father.","1,500g, 1,875g, 2,250g, 3,000g","1,875g, 2,343g, 2,812g, 3,750g","2,250g, 2,812g, 3,375g, 4,500g",East Pier on The Beach. Requires level 5 fishing.,Any,All Seasons,Any,19-21,95 mixed,170,2025-11-25T23:16:43.926Z
Ms. Angler,Uses a bioluminescent dangler to attract prey.,"900g, 1,125g, 1,350g, 1,800g","1,125g, 1,406g, 1,687g, 2,250g","1,350g, 1,687g, 2,025g, 2,700g",North of JojaMart on the wooden plank bridge. Requires level 3 fishing.,Any,All Seasons,Any,17-19,85 smooth,155,2025-11-25T23:16:43.926Z
Legend II,The successor to the original Legend.,"5,000g, 6,250g, 7,500g, 10,000g","6,250g, 7,812g, 9,375g, 12,500g","7,500g, 9,375g, 11,250g, 15,000g",The Mountain Lake near the log. Requires level 10 fishing.,Any,All Seasons,Any,49-51,110 mixed,170,2025-11-25T23:16:43.926Z
Glacierfish Jr.,The original Glacierfish had a son...,"1,000g, 1,250g, 1,500g, 2,000g","1,250g, 1,562g, 1,875g, 2,500g","1,500g, 1,875g, 2,250g, 3,000g",South end of Arrowhead Island in Cindersap Forest. Requires level 6 fishing.,Any,All Seasons,Any,26-28,100 mixed,180,2025-11-25T23:16:43.926Z
Radioactive Carp,A carp that spent one too many years in toxic sludge.,"1,000g, 1,250g, 1,500g, 2,000g","1,250g, 1,562g, 1,875g, 2,500g","1,500g, 1,875g, 2,250g, 3,000g",The Sewers,Any,All Seasons,Any,35-37,80 dart,145,2025-11-25T23:16:43.926Z


In [0]:
# Padronização de tipos e desempacotamento (adição) de colunas.

df_legendary_fishII_detail_silver = split_col(df_legendary_fishII_detail_silver, "difficulty_behavior", 0, " ", "int", "difficulty")
df_legendary_fishII_detail_silver = split_col(df_legendary_fishII_detail_silver, "difficulty_behavior", 1, " ", "string", "behavior").drop("difficulty_behavior")
df_legendary_fishII_detail_silver = min_and_max(df_legendary_fishII_detail_silver, "size_inches", "-", "int", True)
df_legendary_fishII_detail_silver = split_time(df_legendary_fishII_detail_silver, "time", "-", True)
df_legendary_fishII_detail_silver = normalize_prices(df_legendary_fishII_detail_silver, ["name", "description", "location", "time_min", "time_max", "season", "weather", "size_inches_min", "size_inches_max", "difficulty", "behavior", "base_xp", "ingestion_timestamp"])
df_legendary_fishII_detail_silver = reordering_df(df_legendary_fishII_detail_silver, df_legendary_fishII_detail_origin_cols)

df_legendary_fishII_detail_silver.limit(5).display()

name,description,price,fisher_profession_25,angler_profession_50,location,time_min,time_max,season,weather,size_inches_min,size_inches_max,difficulty,behavior,base_xp,ingestion_timestamp
Son of Crimsonfish,"He hatched in the warm summer water, and followed in the footsteps of his father.","1500, 1875, 2250, 3000","1875, 2343, 2812, 3750","2250, 2812, 3375, 4500",East Pier on The Beach. Requires level 5 fishing.,Any,Any,All Seasons,Any,19,21,95,mixed,170,2025-11-25 23:16:45.177292
Ms. Angler,Uses a bioluminescent dangler to attract prey.,"900, 1125, 1350, 1800","1125, 1406, 1687, 2250","1350, 1687, 2025, 2700",North of JojaMart on the wooden plank bridge. Requires level 3 fishing.,Any,Any,All Seasons,Any,17,19,85,smooth,155,2025-11-25 23:16:45.177292
Legend II,The successor to the original Legend.,"5000, 6250, 7500, 10000","6250, 7812, 9375, 12500","7500, 9375, 11250, 15000",The Mountain Lake near the log. Requires level 10 fishing.,Any,Any,All Seasons,Any,49,51,110,mixed,170,2025-11-25 23:16:45.177292
Glacierfish Jr.,The original Glacierfish had a son...,"1000, 1250, 1500, 2000","1250, 1562, 1875, 2500","1500, 1875, 2250, 3000",South end of Arrowhead Island in Cindersap Forest. Requires level 6 fishing.,Any,Any,All Seasons,Any,26,28,100,mixed,180,2025-11-25 23:16:45.177292
Radioactive Carp,A carp that spent one too many years in toxic sludge.,"1000, 1250, 1500, 2000","1250, 1562, 1875, 2500","1500, 1875, 2250, 3000",The Sewers,Any,Any,All Seasons,Any,35,37,80,dart,145,2025-11-25 23:16:45.177292


# Padronizando colunas: df_nightmarketfish_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_nightmarketfish_silver = rename_cols(df_nightmarketfish_silver)
df_nightmarketfish_origin_cols = df_nightmarketfish_silver.columns

df_nightmarketfish_silver.limit(5).display()

name,description,price,fish_profession_25,angler_profession_50,size,difficulty_behavior,base_xp,used_in,ingestion_timestamp
Midnight Squid,A strange and mysterious denizen of the ocean's twilight depths.,"100g, 125g, 150g, 200g","125g, 156g, 187g, 250g","150g, 187g, 225g, 300g",8-25,55 Sinker,21,null,2025-11-25T23:16:46.034Z
Spook Fish,The huge eyes can detect the faint silhouettes of prey.,"220g, 275g, 330g, 440g","275g, 343g, 412g, 550g","330g, 412g, 495g, 660g",8-25,60 Dart,23,null,2025-11-25T23:16:46.034Z
Blobfish,"This odd creature floats above the ocean floor, consuming any edible material in its path.","500g, 625g, 750g, 1,000g","625g, 781g, 937g, 1,250g","750g, 937g, 1,125g, 1,500g",8-25,75 Floater,28,Master Fisher's Bundle,2025-11-25T23:16:46.034Z


In [0]:
# Padronização de tipos e desempacotamento (adição) de colunas.

df_nightmarketfish_silver = split_col(df_nightmarketfish_silver, "difficulty_behavior", 0, " ", "int", "difficulty")
df_nightmarketfish_silver = split_col(df_nightmarketfish_silver, "difficulty_behavior", 1, " ", "string", "behavior").drop("difficulty_behavior")
df_nightmarketfish_silver = min_and_max(df_nightmarketfish_silver, "size", "-", "int", True)
df_nightmarketfish_silver = normalize_prices(df_nightmarketfish_silver, ["name", "description", "location", "size_min", "size_max", "difficulty", "behavior", "base_xp", "used_in", "ingestion_timestamp"])
df_nightmarketfish_silver = reordering_df(df_nightmarketfish_silver, df_nightmarketfish_origin_cols)

df_nightmarketfish_silver.limit(5).display()

name,description,price,fish_profession_25,angler_profession_50,size_min,size_max,difficulty,behavior,base_xp,used_in,ingestion_timestamp
Midnight Squid,A strange and mysterious denizen of the ocean's twilight depths.,"100, 125, 150, 200","125, 156, 187, 250","150, 187, 225, 300",8,25,55,Sinker,21,null,2025-11-25 23:16:47.192684
Spook Fish,The huge eyes can detect the faint silhouettes of prey.,"220, 275, 330, 440","275, 343, 412, 550","330, 412, 495, 660",8,25,60,Dart,23,null,2025-11-25 23:16:47.192684
Blobfish,"This odd creature floats above the ocean floor, consuming any edible material in its path.","500, 625, 750, 1000","625, 781, 937, 1250","750, 937, 1125, 1500",8,25,75,Floater,28,Master Fisher's Bundle,2025-11-25 23:16:47.192684


# Padronizando colunas: df_behavior_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_behavior_silver = rename_cols(df_behavior_silver)

df_behavior_silver.limit(5).display()

behavior,description,ingestion_timestamp
Mixed,These fish use the basic movement pattern.,2025-11-25T23:16:48.053Z
Smooth,These fish have a more steady movement pattern.,2025-11-25T23:16:48.053Z
Sinker,These fish have a faster downwards acceleration.,2025-11-25T23:16:48.053Z
Floater,These fish have a faster upwards acceleration.,2025-11-25T23:16:48.053Z
Dart,"These fish move more randomly with a [2 x difficulty]% higher possible amplitude in both directions, moving erratically up and down the fishing bar.",2025-11-25T23:16:48.053Z


# Padronizando colunas: df_villagers_silver

In [0]:
# Padronização dos nomes das colunas para lower_snake_case.

df_villagers_silver = rename_cols(df_villagers_silver)

df_villagers_silver.limit(5).display()

name,gender,marriage_candidate,giftable,heart_events,ingestion_timestamp
Alex,Male,Yes,Yes,8,2025-11-25T23:16:49.184Z
Elliot,Male,Yes,Yes,7,2025-11-25T23:16:49.184Z
Harvey,Male,Yes,Yes,7,2025-11-25T23:16:49.184Z
Sam,Male,Yes,Yes,8,2025-11-25T23:16:49.184Z
Sebastian,Male,Yes,Yes,7,2025-11-25T23:16:49.184Z


# Salvando os dados na camada Silver
Depois de padronizados e de algumas colunas serem "desempactodas", podemos salvar nossos DataFrames na camada Silver para que prossigamos fazendo refinamentos posteriormente e para garantirmos o acesso aos novos dados modificados de forma segura.

In [0]:
try:
    save_df(df_crabpotandothercatchables_silver, "crabpotandothercatchables", "name")
except Exception as e:
    print("Erro ao salvar a tabela crabpotandothercatchables: ", e)
    raise

tabela crabpotandothercatchables existe


In [0]:
try:
    save_df(df_fish_detail_silver, "fish_detail", "name")
except Exception as e:
    print("Erro ao salvar a tabela fish_detail: ", e)
    raise

tabela fish_detail existe


In [0]:
try:
    save_df(df_fish_price_breakdown_silver, "fish_price_breakdown", "name")
except Exception as e:
    print("Erro ao salvar a tabela fish_price_breakdown: ", e)
    raise

tabela fish_price_breakdown existe


In [0]:
try:
    save_df(df_legendary_fish_detail_silver, "legendary_fish_detail", "name")
except Exception as e:
    print("Erro ao salvar a tabela legendary_fish_detail: ", e)
    raise

tabela legendary_fish_detail existe


In [0]:
try:
    save_df(df_legendary_fish_price_breakdown_silver, "legendary_fish_price_breakdown", "name")
except Exception as e:
    print("Erro ao salvar a tabela legendary_fish_price_breakdown: ", e)
    raise

tabela legendary_fish_price_breakdown existe


In [0]:
try:
    save_df(df_legendary_fishII_detail_silver, "legendary_fishII_detail", "name")
except Exception as e:
    print("Erro ao salvar a tabela legendary_fishII_detail: ", e)
    raise

tabela legendary_fishII_detail existe


In [0]:
try:
    save_df(df_nightmarketfish_silver, "nightmarketfish", "name")
except Exception as e:
    print("Erro ao salvar a tabela nightmarketfish: ", e)
    raise

tabela nightmarketfish existe


In [0]:
try:
    save_df(df_villagers_silver, "villagers", "name")
except Exception as e:
    print("Erro ao salvar a tabela villagers: ", e)
    raise

tabela villagers existe


In [0]:
try:
    save_df(df_behavior_silver, "behavior", "behavior")
except Exception as e:
    print("Erro ao salvar a tabela behavior: ", e)
    raise

tabela behavior existe
